In [22]:
import pandas as pd
import json

In [101]:
with open('2018/201800002') as fd:
    data = json.load(fd)

In [56]:
# tiltle
title = data['abstracts-retrieval-response']['item']['bibrecord']['head']['citation-title']
print(title)

The measurement of drug-induced interferon γ-releasing cells and lymphocyte proliferation in severe cutaneous adverse reactions


In [75]:
# pub_date 
pub_year = data['abstracts-retrieval-response']['item']['bibrecord']['head']['source']['publicationdate']['year']
pub_month = data['abstracts-retrieval-response']['item']['bibrecord']['head']['source']['publicationdate']['month']
pub_day = data['abstracts-retrieval-response']['item']['bibrecord']['head']['source']['publicationdate']['day']
pub_date = pub_day + '/' + pub_month + '/' + pub_year
print(pub_date)

01/06/2018


In [76]:
# abstract 
abstract = data['abstracts-retrieval-response']['item']['bibrecord']['head']['abstracts']
print(abstract)

© 2018 European Academy of Dermatology and VenereologyBackground: The lymphocyte transformation test (LTT) is a standard laboratory method to identify culprit drugs in patients with a history of drug-induced non-immediate hypersensitivity and is mainly performed during the recovery phase. The measurement of drug-specific interferon γ (IFN-γ)-releasing cells has been introduced to confirm culprit drugs, even during the acute phase of drug allergy. Objectives: This study aimed to evaluate the capability of the enzyme-linked immunospot assay (ELISpot) to detect drug-specific IFN-γ-releasing cells during the acute phase and the capability of LTT to identify culprit drugs during the recovery phase in patients presenting with severe cutaneous adverse reactions (SCARs). Methods: Peripheral blood mononuclear cells (PBMCs) from 23 SCAR patients were collected during the acute and recovery phases and assayed for drug-specific IFN-γ-releasing cells and lymphocyte proliferation, respectively. Resu

In [77]:
# language 
language = data['abstracts-retrieval-response']['item']['bibrecord']['head']['citation-info']['citation-language']['@language']
print(language)

English


In [78]:
# refcount  
ref_count = data['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['@refcount']
print(ref_count)

20


In [79]:
# citedcount 
citedby_count = data['abstracts-retrieval-response']['coredata']['citedby-count']
print(citedby_count)

24


In [80]:
# author_id 
authors_list = data['abstracts-retrieval-response']['authors']['author']
author_id = []
for author in authors_list:
    author_id.append(author['@auid'])
print(author_id)

['57189870740', '6504156938', '26532099300', '16403174400', '7003678340', '26024040600', '44561414300', '41261158800', '37017011900', '57190868628', '56871252100', '55418940200', '57209265337', '6508090526']


In [91]:
# classification_code
classification_list = data['abstracts-retrieval-response']['item']['bibrecord']['head']['enhancement']['classificationgroup']['classifications']  
classification_code = []
for c in classification_list:
    for x in c['classification']:
        classification_code.append(x['$'])
    # classification_code.append(c['classification'])
print(classification_code)

KeyError: '$'

In [ ]:
import os
import json
import csv

def safe_get(data, path, default=None):
    """
    Safely retrieves a value from a nested dictionary or list.
    
    :param data: The dictionary or list to retrieve data from.
    :param path: A list of keys/indexes representing the path to the desired value.
    :param default: The value to return if the path does not exist or is invalid.
    :return: The retrieved value or the default value.
    """
    for key in path:
        if isinstance(data, dict):
            data = data.get(key, default)
        elif isinstance(data, list) and isinstance(key, int) and 0 <= key < len(data):
            data = data[key]
        else:
            return default
    return data

def check_slice_or_single(data):
    """
    Ensures that the returned data is always a list, even if it's a single item.
    
    :param data: The data to check, which can be a list or a single value.
    :return: A list of data values.
    """
    if data is None:
        return None
    if isinstance(data, list):
        return data
    return [data]

def get_classification_codes(classification_list):
    """
    Processes the classification list and extracts the relevant classification code.
    
    :param classification_list: List of classifications, can be of different types.
    :return: List of classification codes.
    """
    classification_codes = []
    
    for classification in classification_list:
        if isinstance(classification, dict):
            classification_type = classification.get('@type')
            classification_data = classification.get('classification')
            
            if classification_type == 'ASJC':
                if isinstance(classification_data, list):
                    # For ASJC, extract $ field from all items in the list
                    classification_codes.extend([item.get('$') for item in classification_data if isinstance(item, dict)])
                elif isinstance(classification_data, dict):
                    # For ASJC, extract $ field from single item
                    classification_codes.append(classification_data.get('$'))
            
            elif classification_type == 'SUBJABBR':
                if isinstance(classification_data, list):
                    # For SUBJABBR, extract $ field from all items in the list
                    classification_codes.extend([item.get('$') for item in classification_data if isinstance(item, dict)])
                elif isinstance(classification_data, dict):
                    # For SUBJABBR, extract $ field from single item
                    classification_codes.append(classification_data.get('$'))
            
            elif classification_type in ['CPXCLASS', 'FLXCLASS']:
                if isinstance(classification_data, list):
                    # For CPXCLASS and FLXCLASS, extract classification-code from all items in the list
                    classification_codes.extend([item.get('classification-code') for item in classification_data if isinstance(item, dict)])
                elif isinstance(classification_data, dict):
                    # For CPXCLASS and FLXCLASS, extract classification-code from single item
                    classification_codes.append(classification_data.get('classification-code'))
    
    return classification_codes

folder_path = '2018'
output_csv = 'paper_2019.csv'

columns = [
    'title', 'pub_date', 'abstract', 'language', 'ref_count',
    'citedby_count', 'author_id', 'subject_areas_id', 'keywords',
    'idxterms', 'classification_code'
]

with open(output_csv, mode='w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=columns)
    writer.writeheader()

    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        if not os.path.isfile(file_path):
            continue

        try:
            with open(file_path, 'r', encoding='utf-8') as fd:
                data = json.load(fd)

                title = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'citation-title'], None)
                
                pub_year = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'source', 'publicationdate', 'year'], None)
                pub_month = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'source', 'publicationdate', 'month'], None)
                pub_day = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'source', 'publicationdate', 'day'], None)
                pub_date = f"{pub_day}/{pub_month}/{pub_year}" if pub_year and pub_month and pub_day else None

                abstract = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'abstracts'], None)

                language = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'citation-info', 'citation-language', '@language'], None)

                ref_count = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'tail', 'bibliography', '@refcount'], None)

                citedby_count = safe_get(data, ['abstracts-retrieval-response', 'coredata', 'citedby-count'], None)

                authors_list = safe_get(data, ['abstracts-retrieval-response', 'authors', 'author'], None)
                author_id = [author.get('@auid', None) for author in authors_list if isinstance(author, dict)] if authors_list else None

                subject_areas_list = safe_get(data, ['abstracts-retrieval-response', 'subject-areas', 'subject-area'], None)
                subject_areas_id = [subject_area.get('@code', None) for subject_area in subject_areas_list if isinstance(subject_area, dict)] if subject_areas_list else None

                authkeywords = safe_get(data, ['abstracts-retrieval-response', 'citation-info', 'author-keywords', 'author-keyword'], None)
                # Check if authkeywords is not None and handle both single and multiple keywords
                if authkeywords:
                    if isinstance(authkeywords, dict):
                        # Single author-keyword in File 1, wrap it in a list
                        keywords = [authkeywords.get('$', None)]
                    else:
                        # Multiple author-keywords in File 2
                        keywords = [k.get('$', None) for k in authkeywords]
                else:
                    keywords = None
                keywords = check_slice_or_single(keywords)

                idxterms_data = safe_get(data, ['abstracts-retrieval-response', 'idxterms'], None)
                if isinstance(idxterms_data, dict):  # Handle case when idxterms is a dictionary
                    mainterm = idxterms_data.get('mainterm')
                    idxterms = [mainterm.get('$', None)] if isinstance(mainterm, dict) else None
                elif isinstance(idxterms_data, list):  # Handle case when idxterms is a list
                    idxterms = [i.get('$', None) for i in idxterms_data.get('mainterm', [])] if idxterms_data else None
                else:
                    idxterms = None

                classification_list = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'enhancement', 'classificationgroup', 'classifications'], None)
                classification_code = get_classification_codes(classification_list)
                classification_code = check_slice_or_single(classification_code)

                writer.writerow({
                    'title': title,
                    'pub_date': pub_date,
                    'abstract': abstract,
                    'language': language,
                    'ref_count': ref_count,
                    'citedby_count': citedby_count,
                    'author_id': author_id,
                    'subject_areas_id': subject_areas_id,
                    'keywords': keywords,
                    'idxterms': idxterms,
                    'classification_code': classification_code
                })
        except Exception as e:
            print(f"Error processing file {filename}: {e}")

print(f"Data extraction complete. Results saved to {output_csv}.")


Data extraction complete. Results saved to paper_2019.csv.


In [32]:
with open('2018/201800036') as fd:
    data4 = json.load(fd)

In [84]:
# subject_areas_id
subject_areas_list = data['abstracts-retrieval-response']['subject-areas']['subject-area']
subject_area_id = []
for subject_area in subject_areas_list:
    subject_area_id.append(subject_area['@code'])
print(subject_area_id)


['2708', '2725']


In [70]:
keywords = data['abstracts-retrieval-response']['authkeywords']['author-keyword'][0]['$']
print(keywords)

TypeError: 'NoneType' object is not subscriptable

In [102]:
# keywords
keywords = None
if data['abstracts-retrieval-response']['authkeywords'] == None:
    keywords = None
else:
    keywords = []
    for k in data['abstracts-retrieval-response']['authkeywords']['author-keyword']:
        keywords.append(k['$'])
print(keywords)

['Circulating fluidized bed', 'Computational fluid dynamics', 'Multiphase flow models', 'Riser', 'Sorption enhanced steam methane reforming']


In [47]:
with open('2018/201800479') as fd:
    data2 = json.load(fd)

In [37]:
with open('2018/201800017') as fd:
    data3 = json.load(fd)

In [39]:
a = data3['abstracts-retrieval-response']['idxterms']['mainterm'][0]['$']
b = data['abstracts-retrieval-response']['idxterms']
print(a)
print(b)

Composite photocatalysts
None


In [103]:
# idxterms
idxterms = None
if data['abstracts-retrieval-response']['idxterms'] == None:
    idxterms = None
else:
    idxterms = []
    for i in data['abstracts-retrieval-response']['idxterms']['mainterm']:
        idxterms.append(i['$'])
print(idxterms)

['Circulating fluidized bed', 'Circulating fluidized bed riser', 'Full factorial design', 'Multi-phase flow models', 'Ni-based catalyst', 'Radial distributions', 'Reaction parameters', 'Riser']


In [114]:
import os
import json
import csv

def safe_get(data, path, default=None):
    """
    Safely retrieves a value from a nested dictionary or list.
    
    :param data: The dictionary or list to retrieve data from.
    :param path: A list of keys/indexes representing the path to the desired value.
    :param default: The value to return if the path does not exist or is invalid.
    :return: The retrieved value or the default value.
    """
    for key in path:
        if isinstance(data, dict):
            data = data.get(key, default)
        elif isinstance(data, list) and isinstance(key, int) and 0 <= key < len(data):
            data = data[key]
        else:
            return default
    return data

def check_slice_or_single(data):
    """
    Ensures that the returned data is always a list, even if it's a single item.
    
    :param data: The data to check, which can be a list or a single value.
    :return: A list of data values.
    """
    if data is None:
        return None
    if isinstance(data, list):
        return data
    return [data]

def get_classification_codes(classification_list):
    """
    Processes the classification list and extracts the relevant classification code.
    
    :param classification_list: List of classifications, can be of different types.
    :return: List of classification codes.
    """
    classification_codes = []
    
    for classification in classification_list:
        if isinstance(classification, dict):
            classification_type = classification.get('@type')
            classification_data = classification.get('classification')
            
            if classification_type == 'ASJC':
                # For ASJC, classification might be a single string or a list
                if isinstance(classification_data, list):
                    classification_codes.extend([item.get('$') for item in classification_data if isinstance(item, dict)])
                else:
                    classification_codes.append(classification_data)  # Single string classification
            
            elif classification_type == 'SUBJABBR':
                # For SUBJABBR, classification might be a single string or a list
                if isinstance(classification_data, list):
                    classification_codes.extend([item.get('$') for item in classification_data if isinstance(item, dict)])
                else:
                    classification_codes.append(classification_data)  # Single string classification
            
            elif classification_type in ['CPXCLASS', 'FLXCLASS']:
                if isinstance(classification_data, list):
                    classification_codes.extend([item.get('classification-code') for item in classification_data if isinstance(item, dict)])
                elif isinstance(classification_data, dict):
                    classification_codes.append(classification_data.get('classification-code'))
    
    return classification_codes


folder_path = '2023'
output_csv = 'paper_2023.csv'

columns = [
    'pid','title', 'pub_date', 'abstract', 'language', 'ref_count',
    'citedby_count', 'author_id', 'subject_areas_id', 'keywords',
    'idxterms', 'classification_code'
]

with open(output_csv, mode='w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=columns)
    writer.writeheader()

    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        if not os.path.isfile(file_path):
            continue

        try:
            with open(file_path, 'r', encoding='utf-8') as fd:
                data = json.load(fd)

                title = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'citation-title'], None)
                
                pub_year = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'source', 'publicationdate', 'year'], None)
                pub_month = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'source', 'publicationdate', 'month'], None)
                pub_day = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'source', 'publicationdate', 'day'], None)
                pub_date = f"{pub_day}/{pub_month}/{pub_year}" if pub_year and pub_month and pub_day else None

                abstract = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'abstracts'], None)

                language = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'citation-info', 'citation-language', '@language'], None)

                ref_count = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'tail', 'bibliography', '@refcount'], None)

                citedby_count = safe_get(data, ['abstracts-retrieval-response', 'coredata', 'citedby-count'], None)

                authors_list = safe_get(data, ['abstracts-retrieval-response', 'authors', 'author'], None)
                author_id = [author.get('@auid', None) for author in authors_list if isinstance(author, dict)] if authors_list else None

                subject_areas_list = safe_get(data, ['abstracts-retrieval-response', 'subject-areas', 'subject-area'], None)
                subject_areas_id = [subject_area.get('@code', None) for subject_area in subject_areas_list if isinstance(subject_area, dict)] if subject_areas_list else None

                # Keywords extraction (Fix applied here to match your working example)
                authkeywords = safe_get(data, ['abstracts-retrieval-response', 'authkeywords'], None)
                keywords = None
                if authkeywords and 'author-keyword' in authkeywords:
                    keywords = [k['$'] for k in authkeywords['author-keyword'] if isinstance(k, dict)]
                keywords = check_slice_or_single(keywords)

                # Handling idxterms
                idxterms_data = safe_get(data, ['abstracts-retrieval-response', 'idxterms'], None)
                if idxterms_data:
                    if isinstance(idxterms_data, dict):  # If idxterms is a dictionary
                        mainterm = idxterms_data.get('mainterm')
                        if isinstance(mainterm, list):
                            idxterms = [i.get('$', None) for i in mainterm]  # Extract the '$' value from each item in the list
                        elif isinstance(mainterm, dict):
                            idxterms = [mainterm.get('$', None)]  # If 'mainterm' is a single dict, extract the '$'
                        else:
                            idxterms = None
                    elif isinstance(idxterms_data, list):  # If idxterms is a list
                        idxterms = [i.get('$', None) for i in idxterms_data if isinstance(i, dict)]  # Loop through and extract '$'
                    else:
                        idxterms = None
                else:
                    idxterms = None

                # Extracting classification codes
                classification_list = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'enhancement', 'classificationgroup', 'classifications'], None)
                classification_code = get_classification_codes(classification_list)
                classification_code = check_slice_or_single(classification_code)

                writer.writerow({
                    'pid': filename,
                    'title': title,
                    'pub_date': pub_date,
                    'abstract': abstract,
                    'language': language,
                    'ref_count': ref_count,
                    'citedby_count': citedby_count,
                    'author_id': author_id,
                    'subject_areas_id': subject_areas_id,
                    'keywords': keywords,
                    'idxterms': idxterms,
                    'classification_code': classification_code
                })
        except Exception as e:
            print(f"Error processing file {filename}: {e}")

print(f"Data extraction complete. Results saved to {output_csv}.")


Data extraction complete. Results saved to paper_2023.csv.
